# 작사가 만들기

## 데이터 준비


In [1]:
import os, re 
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

raw_corpus = []
path_folder = './data/lyrics/'
path_files = glob.glob(path_folder + '*.txt')

for path_file in path_files:
    with open(path_file, 'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 데이터 전처리

### 샘플 정제

In [2]:
# 1. 단어가 없는 문장 제거
# 2. 동일 단어 인식 (소문자로 인식) 
# 3. 특수문자 분리
# 4. 여러공백 하나로 처리
# 5. <start>, <end> 추가
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


In [3]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

### 토큰화, 시퀀스로 변환(tensor)

In [4]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # 출연하는 단어들에 숫자를 매핑 한다.
    tokenizer.fit_on_texts(corpus)
    # 단어장을 사용하여 문장을 매핑된 순자로 바꾸고, 리스트 형식으로 변환하여 Tensor로 만든다.
    tensor = tokenizer.texts_to_sequences(corpus)
    # 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기
    idx_tensor = []
    for idx, sentence in enumerate(tensor):
        if len(sentence) > 15:
            idx_tensor.append(idx)
    idx_tensor = idx_tensor[::-1]
    
    for idx in idx_tensor:
        del tensor[idx]
        del corpus[idx]

    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    word_index = tokenizer.word_index
    return tensor, tokenizer

In [5]:
tensor, tokenizer = tokenize(corpus)


#### 토큰, 시퀀스 확인하기 (사전)


In [6]:

print('token')
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break
        
print('\n')
print('copus[0]\n', corpus[:1])
print('\n')
print('tensor[0]\n', tensor[:1])

token
1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


copus[0]
 ['<start> now i ve heard there was a secret chord <end>']


tensor[0]
 [[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
     0]]


## train data 생성, sample, label 생성

언어 모델의 입력 문장 :  <start> 나는 밥을 먹었다
언어 모델의 출력 문장 : 나는 밥을 먹었다 <end>

In [7]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 1)


In [9]:
print('\ntensor: ', len(tensor))

print('\nenc_train: ', enc_train.shape)
print('enc_val: ', enc_val.shape)
print('dec_train: ', dec_train.shape)
print('dec_val: ', dec_val.shape)

print(tensor[:3, :10])

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break


tensor:  156013

enc_train:  (124810, 14)
enc_val:  (31203, 14)
dec_train:  (124810, 14)
dec_val:  (31203, 14)
[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]]
1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


### batch 생성

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# src_input array와 tgt_intput array 매칭
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
# 무작위로 섞은 BUFFER_SIZE개짜리 데이터셋 생성 
dataset = dataset.shuffle(BUFFER_SIZE)
# BATCH_SIZE 만큼씩 나눠서 Dataset 생성, drop_remainder=True는 나누다 남은것을 버리지 않고 Dataset으로 남긴다
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


## 모델 설계


In [11]:
 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
src_sample = None
tgt_sample = None
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.01212951e-04,  1.89399725e-04,  2.05378718e-04, ...,
         -1.26890547e-04,  1.55438320e-04,  2.35424432e-05],
        [-1.65608595e-04,  2.19021313e-04,  5.94105630e-04, ...,
          7.34932255e-05,  2.14834610e-04, -2.44826224e-04],
        [-2.90239434e-04,  2.42251976e-04,  6.02376414e-04, ...,
          2.84549635e-04,  2.75603699e-04, -4.55546222e-04],
        ...,
        [ 2.73650308e-04,  9.89516266e-04,  2.49688677e-03, ...,
         -7.38517730e-04, -1.75156491e-03, -1.08589313e-03],
        [ 4.42998338e-04,  9.43913474e-04,  2.28636758e-03, ...,
         -6.50344358e-04, -1.47134683e-03, -1.24295824e-03],
        [ 5.07265329e-04,  8.58019688e-04,  2.08122865e-03, ...,
         -4.74725122e-04, -7.84911623e-04, -1.34668220e-03]],

       [[-1.01212951e-04,  1.89399725e-04,  2.05378718e-04, ...,
         -1.26890547e-04,  1.55438320e-04,  2.35424432e-05],
        [ 1.99995076e-04,  1.41418990e-04,  4

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
609/609 [==============================] - 98s 154ms/step - loss: 3.4076
Epoch 2/10
609/609 [==============================] - 103s 168ms/step - loss: 2.9621
Epoch 3/10
609/609 [==============================] - 103s 168ms/step - loss: 2.7871
Epoch 4/10
609/609 [==============================] - 103s 169ms/step - loss: 2.6552
Epoch 5/10
609/609 [==============================] - 103s 168ms/step - loss: 2.5436
Epoch 6/10
609/609 [==============================] - 103s 168ms/step - loss: 2.4439
Epoch 7/10
609/609 [==============================] - 103s 168ms/step - loss: 2.3524
Epoch 8/10
609/609 [==============================] - 103s 169ms/step - loss: 2.2667
Epoch 9/10
609/609 [==============================] - 103s 169ms/step - loss: 2.1859
Epoch 10/10
609/609 [==============================] - 103s 168ms/step - loss: 2.1095


loss : 2.1095

## test 생성

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다
    print(test_tensor[0].numpy())
    for word_index in test_tensor[0].numpy():
        print(word_index)
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text(model, tokenizer, init_sentence="<start> i")

[   2    5   22    9  158   80    4    5   22    9 5120    3]
2
5
22
9
158
80
4
5
22
9
5120
3


'<start> i m a bad girl , i m a cunt <end> '

---

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다
    print(test_tensor[0].numpy())
    for word_index in test_tensor[0].numpy():
        print(word_index)
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

[ 2  5 33  7  4  5 33  7  3]
2
5
33
7
4
5
33
7
3


'<start> i love you , i love you <end> '

최종 loss : 2.1095 <br>
입력 sentence : 'i love'<br>
출력 sentence : '<start> i love you , i love you <end> '<br>